In [2]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [4]:
query = "SELECT * FROM `basedosdados.world_gsps_consortium_gsps.global_indicators`"

In [5]:
## df da base dos dados

df=bd.read_sql(query, billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 152275/152275 [00:25<00:00, 5878.89rows/s]


In [5]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\GSPS")

In [6]:
df.to_csv("GSPS_indicadores.csv")

NameError: name 'df' is not defined

In [8]:
## Esse aqui pega do original que está na BD
df1 = pd.read_csv("GSPS_indicadores.csv")
df1

,Unnamed: 0,year,country,institutions_number,respondents,response_rate,section_org,topic,indicator,group,category,mode,question_text,harmonize,scale,mean,lower_ci,upper_ci,source
0,0,2019,Australia,102,104471,77%,General Impressions: Current Job,General Impressions: Current Job,My job gives me a feeling of personal accompli...,All,All,Online,My job gives me a feeling of personal accompli...,0,1--5,0.727,0.725,0.730,https://www.apsc.gov.au/initiatives-and-progra...
1,1,2019,Australia,102,104471,77%,General Impressions: Current Job,General Impressions: Current Job,My job gives me a feeling of personal accompli...,Gender,Male,Online,My job gives me a feeling of personal accompli...,0,1--5,0.725,0.720,0.729,https://www.apsc.gov.au/initiatives-and-progra...
2,2,2019,Australia,102,104471,77%,General Impressions: Current Job,General Impressions: Current Job,My job gives me a feeling of personal accompli...,Gender,Female,Online,My job gives me a feeling of personal accompli...,0,1--5,0.742,0.739,0.745,https://www.apsc.gov.au/initiatives-and-progra...
3,3,2019,Australia,102,104471,77%,General Impressions: Current Job,General Impressions: Current Job,My job gives me a feeling of personal accompli...,Manager status,Manager,Online,My job gives me a feeling of personal accompli...,0,1--5,0.809,0.805,0.814,https://www.apsc.gov.au/initiatives-and-progra...
4,4,2019,Australia,102,104471,77%,General Impressions: Current Job,General Impressions: Current Job,My job gives me a feeling of personal accompli...,Manager status,Not manager,Online,My job gives me a feeling of personal accompli...,0,1--5,0.687,0.683,0.690,https://www.apsc.gov.au/initiatives-and-progra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152270,152270,2021,Uruguay,20,10232,29%,Work effort and workload,Work effort and workload,Thinking about your first job in the public se...,Region,Salto,Online,Thinking about your first job in the public se...,0,0--1,0.234,0.225,0.244,https://www.gub.uy/oficina-nacional-servicio-c...
152271,152271,2021,Uruguay,20,10232,29%,Work effort and workload,Work effort and workload,Thinking about your first job in the public se...,Region,Durazno,Online,Thinking about your first job in the public se...,0,0--1,0.133,0.125,0.142,https://www.gub.uy/oficina-nacional-servicio-c...
152272,152272,2021,Uruguay,20,10232,29%,Work effort and workload,Work effort and workload,Thinking about your first job in the public se...,Region,Cerro Largo,Online,Thinking about your first job in the public se...,0,0--1,0.161,0.152,0.170,https://www.gub.uy/oficina-nacional-servicio-c...
152273,152273,2021,Uruguay,20,10232,29%,Work effort and workload,Work effort and workload,Thinking about your first job in the public se...,Region,Rivera,Online,Thinking about your first job in the public se...,0,0--1,0.125,0.117,0.134,https://www.gub.uy/oficina-nacional-servicio-c...


In [9]:
paises = df1[(df1['topic']=="Job satisfaction") & (df1['group']=="All")][['country',"mean"]]

In [19]:
df[df['topic'].str.startswith('Job')]['topic'].unique()

array(['Job stability', 'Job characteristics', 'Job security',
       'Job satisfaction'], dtype=object)

In [ ]:
## Explorando o original
## Os países Romênia, Colômbia, Nigéria, Chile, Ucrânia, Lituânia, Uruguai, Eslováquia, Austrália e Estados Unidos tiveram seus valores coletadas apenas do dashboard disponibilizado 

In [ ]:
## Trabalhar apenas com satisfação no trabalho, por enquanto. 

In [10]:
## Esse aqui é referente ao gráfico no tableau
#https://public.tableau.com/app/profile/global.survey.of.public.servants/viz/GlobalSurveyofPublicServants_16357969958320/GlobalPublicServantIndicators

df = pd.read_csv("satisfacao_trabalho.csv")

In [11]:
df2=df.merge(paises,right_on="country", left_on="Country",how='left')

In [49]:
df2.columns

Index(['Country', 'CI Length', 'Indicator', 'Institutions number', 'Mode',
       'Question text', 'Respondents', 'Response rate', 'Scale', 'Year',
       'Overall lower', 'Overall upper', 'Overall', 'media', 'country',
       'mean'],
      dtype='object')

In [12]:
df2['Overall'] = df2['Overall'].str.replace("%", "")
df2['Overall'] = pd.to_numeric(df2['Overall'])


In [13]:
df2['Overall'] = df2['Overall']/100

In [14]:
# Definir a quantidade de casas decimais desejada
casas_decimais = 4

# Adicionar zeros à direita na coluna
df2['Overall'] = df2['Overall'].apply(lambda x: '{:.{prec}f}'.format(x, prec=casas_decimais))

In [15]:
df2['media']= np.where(df2['Country'].isin(paises['country']),df2['mean'],df2['Overall'])

In [88]:
df2.columns

Index(['Country', 'CI Length', 'Indicator', 'Institutions number', 'Mode',
       'Question text', 'Respondents', 'Response rate', 'Scale', 'Year',
       'Overall lower', 'Overall upper', 'Overall', 'media', 'country', 'mean',
       'numero_decimal'],
      dtype='object')

In [16]:
df2 = df2[['Country', 'CI Length', 'Indicator', 'Institutions number', 'Mode',
       'Question text', 'Respondents', 'Response rate', 'Scale', 'Year',
       'Overall lower', 'Overall upper',  'media']]

In [19]:
df2['media']=pd.to_numeric(df2['media'])

In [3]:
client = bigquery.Client()
dataset_ref = client.dataset('performance_valorizacao_engajamento')

In [26]:
df2['Country'].unique()

array(['Brazil', 'Croatia', 'United States', 'Kosovo', 'Australia',
       'Slovakia', 'Uruguay', 'Albania', 'Estonia', 'Lithuania',
       'Ukraine', 'Chile', 'Nigeria', 'Colombia', 'Romania'], dtype=object)

In [23]:
schema=[bigquery.SchemaField('Country','STRING',description='País'),
bigquery.SchemaField('CI Length','float',description=''),
bigquery.SchemaField('Indicator','STRING',description=''),
bigquery.SchemaField('Institutions number','integer',description=''),
bigquery.SchemaField('Mode','STRING',description=''),
bigquery.SchemaField('Question text','STRING',description=''),
bigquery.SchemaField('Respondents','STRING',description=''),
bigquery.SchemaField('Response rate','STRING',description=''),
 bigquery.SchemaField('Scale','STRING',description=''),
 bigquery.SchemaField('Year','integer',description=''),
 bigquery.SchemaField('Overall lower','STRING',description=''),
 bigquery.SchemaField('Overall upper','STRING',description='P'),
 bigquery.SchemaField('media','float',description='Média do indicador')]

In [24]:
table_ref = dataset_ref.table('GSPS_satisfacao_trabalho') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df2, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=f52c69d1-16c9-4b56-a947-802c82ef3529>